5-2강. 머신러닝 모델 활용 텍스트 분류
=====
요즘IT 주제 예측
-----
지난 시간에 이어 이번에는 다중 클래스 텍스트 분류를 해보겠습니다.<br>
이번에 살펴볼 모델은 나이브 베이즈 분류기와 서포트 벡터 머신입니다.

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

path = '/content/drive/MyDrive/[deep daiv.] 머신러닝 입문/[deep daiv.] 머신러닝 5주차'
filename = '요즘IT_2023.04.27_전처리.csv'
df = pd.read_csv(os.path.join(path, filename))

In [ ]:
df.head(10)

In [ ]:
df.info()

In [ ]:
!pip install konlpy

from konlpy.tag import Okt
okt = Okt()

In [ ]:
# df['본문_전처리'] = df['본문'].map(okt.nouns) # 시간이 약 7분 정도 소요되어 오래 걸려 미리 작업해두었습니다.

In [ ]:
df['본문_전처리'] = df['본문_전처리'].map(' '.join)

In [ ]:
# 결측치 확인
df[df['본문_전처리'].isnull()]

In [ ]:
df.dropna(inplace = True)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB # 다중 Naive Bayes 분류기
from sklearn.metrics import accuracy_score

## Naive Bayes

In [ ]:
# 데이터 스플릿(훈련 데이터와 테스트 데이터)
train_texts, test_texts, train_labels, test_labels = train_test_split(df['본문_전처리'], df['분류'], test_size=0.3, random_state=0)

In [ ]:
# TF-IDF 벡터화
vectorizer = TfidfVectorizer(min_df = 10, ngram_range = (1,2))
train_features = vectorizer.fit_transform(train_texts)
print(train_features)

In [ ]:
pd.DataFrame(train_features.todense())

In [ ]:
# Naive Bayes 모델
nb = MultinomialNB(alpha = 0.01) # alpha는 hyper parameter로 사람이 직접 지정해줘야 합니다.

In [ ]:
# Naive Bayes 모델 훈련 데이터로 학습
nb.fit(train_features, train_labels)

In [ ]:
# 테스트 데이터 벡터화
test_features = vectorizer.transform(test_texts)

In [ ]:
pd.DataFrame(test_features.todense())

In [ ]:
# Naive Bayes 모델로 분류 예측하기
nb.predict(test_features)

In [ ]:
# 분류 예측 결과 확인
pred_labels = nb.predict(test_features)
print('잘못 분류된 아티클: {} out of {}'.format((pred_labels != test_labels).sum(),len(test_labels)))
print('정확도: %.2f' % accuracy_score(test_labels, pred_labels))

In [ ]:
test_df = pd.DataFrame({'본문_전처리' : test_texts, '분류_예측' : pred_labels}); test_df

In [ ]:
test_df = df[['제목', '작성자', '분류', '본문', '본문_전처리']].merge(test_df, on = ['본문_전처리']).copy()

In [ ]:
test_df[test_df['분류_예측'] != test_df['분류']]

In [ ]:
def tokenizer(text):
    word_list = okt.pos(text)
    pos_list = ['Noun', 'Alpha']
    pos_filtered_word_list = [word[0] for word in word_list if word[1] in pos_list]

    return pos_filtered_word_list

In [ ]:
train_texts, test_texts, train_labels, test_labels = train_test_split(df['본문'], df['분류'], test_size=0.3, random_state=0)

# TF-IDF 벡터화
vectorizer = TfidfVectorizer(min_df = 10, ngram_range = (1,2), tokenizer = tokenizer) # 약 5분 소요
train_features = vectorizer.fit_transform(train_texts)
print(train_features)

In [ ]:
# Naive Bayes 모델
nb = MultinomialNB(alpha = 0.01)

# Naive Bayes 모델 훈련 데이터로 학습
nb.fit(train_features, train_labels) # 약 2분 소요

# 테스트 데이터 벡터화
test_features = vectorizer.transform(test_texts)

# 분류 예측 결과 확인
pred_labels = nb.predict(test_features)
print('잘못 분류된 아티클: {} out of {}'.format((pred_labels != test_labels).sum(),len(test_labels)))
print('정확도: %.2f' % accuracy_score(test_labels, pred_labels))